
# Como trasnformar em um arquivo .shp e le ele
- geo_dados.to_file('DATASET.shp')
- rj.to_file('RJ-MUNIC.shp')
- rj_munic = gpd.read_file("RJ-MUNIC.shp")

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
from shapely import Point

# Criação das variáveis com o caminho dos arquivos 

In [ ]:

url = "https://github.com/allanspadini/curso_geopandas/raw/main/dados/RJ_Municipios_2022.shp"
url_csv = "https://raw.githubusercontent.com/allanspadini/curso_geopandas/main/dados/dados.csv"
url_m= "https://github.com/allanspadini/curso_geopandas/raw/main/dados/metro.geojson"
url_t= "https://github.com/allanspadini/curso_geopandas/raw/main/dados/trem.geojson"
url_b= "https://github.com/allanspadini/curso_geopandas/raw/main/dados/brt.geojson"
# tem que baixar o arquivo :https://www.data.rio/datasets/cobertura-vegetal-e-uso-da-terra-2018/explore
url_p = "Cobertura_Vegetal_e_Uso_da_Terra_2018.geojson"#(mude para o caminho do na sua maquina)


# Leitura/importação dos arquivos

In [ ]:
rj = gpd.read_file(url)
df= pd.read_csv(url_csv, sep='\t')
metro= gpd.read_file(url_m)
trem = gpd.read_file(url_t)
brt = gpd.read_file(url_b)
praias = gpd.read_file(url_p)

# Filtragem 

In [ ]:
rj = rj[rj["NM_MUN"] == "Rio de Janeiro"]
praias = praias[praias["classe"]=="Praia"]



# Transformando de DataFrame para GeoDataFrame

In [ ]:
ponto_geometricos = [Point(x) for x in zip(df["Longitude"], df["Latitude"])]
crs = { "proj": "latlong", "ellps":"WGS84", "datum":"WGS84", "no_defs":True}
geo_dados=gpd.GeoDataFrame(df, crs = crs, geometry=ponto_geometricos)


# Padronizando o crs

In [ ]:
rj = rj.to_crs("+proj=utm +zone=23 +south +ellps=GRS80 +units=km +no_defs +type=crs")
geo_dados = geo_dados.to_crs("+proj=utm +zone=23 +south +ellps=GRS80 +units=km +no_defs +type=crs")
metro= metro.to_crs("+proj=utm +zone=23 +south +ellps=GRS80 +units=km +no_defs +type=crs")
trem= trem.to_crs("+proj=utm +zone=23 +south +ellps=GRS80 +units=km +no_defs +type=crs") 
brt = brt.to_crs("+proj=utm +zone=23 +south +ellps=GRS80 +units=km +no_defs +type=crs")  
praias = praias.to_crs("+proj=utm +zone=23 +south +ellps=GRS80 +units=km +no_defs +type=crs")


# Localizando os dados e fazendo a correlação entre eles

In [ ]:

geo_dados = geo_dados[geo_dados['geometry'].within(rj.iloc[0].geometry)]
trem= trem[trem.within(rj.iloc[0].geometry)]
brt= brt[brt.within(rj.iloc[0].geometry)]
geo_dados["Dist_metro"] = geo_dados["geometry"].apply(lambda x: metro.distance(x).min())
geo_dados["Dist_Praia"] = geo_dados["geometry"].apply(lambda x:praias.distance(x).min())
geo_dados_num = geo_dados.select_dtypes(include=[np.number])
correlacao = geo_dados_num.corr()

# Criação dos gráficos a baixo

In [ ]:
base= rj.plot(color="white", edgecolor="black")
geo_dados.plot(ax=base, alpha=0.2)
metro.plot(ax=base, color="#b60a1c", markersize=5)
trem.plot(ax=base, color="#e39143", markersize=5)
brt.plot(ax=base, color="#309143", markersize=5)

## Região de praias

In [ ]:
base= rj.plot(alpha= 0.7)
praias.plot(ax=base, color= "white", edgecolor="black")

## Mapa de Calor

In [ ]:

sns.heatmap(correlacao, annot=True, cmap="RdYlBu", linewidth=0.5)


## Gráfico de Dispersão

In [ ]:
sns.scatterplot(x= "Dist_Praia", y= "Valor",data= geo_dados)